# 03c: Classification LAB - Loan Default Prediction

## วัตถุประสงค์การเรียนรู้
- ใช้ข้อมูล LoanStats ในการทำนายความเสี่ยงเครดิต
- ฝึกฝน Data preprocessing สำหรับ ML
- เปรียบเทียบ Classification algorithms
- ใช้ Cross-validation และ Hyperparameter tuning

---

## 1. โหลดและสำรวจข้อมูล

### ข้อมูล LoanStats
- ข้อมูลสินเชื่อส่วนบุคคล
- เป้าหมาย: ทำนายว่าลูกค้าจะผิดนัดชำระหรือไม่
- Features: รายได้, อายุงาน, จำนวนเงินกู้, etc.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import warnings
warnings.filterwarnings('ignore')

# โหลดข้อมูล
# ใน Google Colab ให้ upload ไฟล์ก่อน
# from google.colab import files
# uploaded = files.upload()

# สำหรับ local environment
loan_data = pd.read_csv('../datasets/LoanStats_web_14422.csv')

print(f"ข้อมูลมี {loan_data.shape[0]} แถว และ {loan_data.shape[1]} คอลัมน์")
print("\nคอลัมน์ทั้งหมด:")
print(loan_data.columns.tolist())

In [ ]:
# ดูข้อมูลเบื้องต้น
print("ข้อมูล 5 แถวแรก:")
print(loan_data.head())

print("\nข้อมูลพื้นฐาน:")
print(loan_data.info())

print("\nข้อมูลที่หายไป:")
missing_data = loan_data.isnull().sum()
print(missing_data[missing_data > 0].head(10))

## 2. สร้าง Target Variable

### กำหนด Default
- ใช้คอลัมน์ 'loan_status' 
- Default = 1: Charged Off, Late, In Grace Period
- Good = 0: Fully Paid, Current

In [ ]:
# ดู loan_status ที่มี
print("Loan Status ทั้งหมด:")
print(loan_data['loan_status'].value_counts())

# สร้าง target variable
# Default = 1, Good = 0
default_status = ['Charged Off', 'Default', 'Late (31-120 days)', 
                 'Late (16-30 days)', 'In Grace Period']

loan_data['is_default'] = loan_data['loan_status'].apply(
    lambda x: 1 if x in default_status else 0
)

print("\nTarget Variable:")
print(loan_data['is_default'].value_counts())
print(f"Default Rate: {loan_data['is_default'].mean():.2%}")

## 3. Feature Selection และ Data Cleaning

### เลือก Features ที่สำคัญ
- ข้อมูลทางการเงิน
- ข้อมูลส่วนตัว
- ประวัติเครดิต

In [ ]:
# เลือก features ที่สำคัญ
important_features = [
    'loan_amnt',           # จำนวนเงินกู้
    'int_rate',            # อัตราดอกเบี้ย
    'installment',         # ค่าผ่อนรายเดือน
    'annual_inc',          # รายได้ต่อปี
    'dti',                 # debt-to-income ratio
    'emp_length',          # อายุงาน
    'home_ownership',      # สถานะที่อยู่อาศัย
    'verification_status', # สถานะการตรวจสอบ
    'purpose',             # วัตถุประสงค์สินเชื่อ
    'grade',               # เกรดสินเชื่อ
    'is_default'           # target
]

# สร้าง dataset ใหม่
df = loan_data[important_features].copy()

print(f"Dataset ใหม่มี {df.shape[0]} แถว และ {df.shape[1]} คอลัมน์")
print("\nข้อมูลที่หายไป:")
print(df.isnull().sum())

In [ ]:
# ทำความสะอาดข้อมูล

# 1. ลบแถวที่มีข้อมูลสำคัญหายไป
df = df.dropna(subset=['annual_inc', 'dti', 'int_rate'])

# 2. จัดการ emp_length
df['emp_length'] = df['emp_length'].fillna('Unknown')
# แปลง emp_length เป็นตัวเลข
def convert_emp_length(emp_len):
    if pd.isna(emp_len) or emp_len == 'Unknown':
        return 0
    elif '< 1' in str(emp_len):
        return 0.5
    elif '10+' in str(emp_len):
        return 10
    else:
        return float(str(emp_len).split()[0])

df['emp_length_years'] = df['emp_length'].apply(convert_emp_length)

# 3. ทำ LabelEncoding สำหรับ categorical variables
categorical_cols = ['home_ownership', 'verification_status', 'purpose', 'grade']
label_encoders = {}

for col in categorical_cols:
    if col in df.columns:
        le = LabelEncoder()
        df[col + '_encoded'] = le.fit_transform(df[col].astype(str))
        label_encoders[col] = le

print(f"Dataset หลังทำความสะอาด: {df.shape}")
print("\nข้อมูลที่หายไป:")
print(df.isnull().sum())

## 4. Exploratory Data Analysis

In [ ]:
# วิเคราะห์ความสัมพันธ์ระหว่าง features กับ target
fig, axes = plt.subplots(2, 3, figsize=(18, 10))

# 1. Loan Amount vs Default
axes[0,0].boxplot([df[df['is_default']==0]['loan_amnt'], 
                   df[df['is_default']==1]['loan_amnt']], 
                  labels=['Good', 'Default'])
axes[0,0].set_title('Loan Amount vs Default')
axes[0,0].set_ylabel('Loan Amount')

# 2. Interest Rate vs Default
axes[0,1].boxplot([df[df['is_default']==0]['int_rate'], 
                   df[df['is_default']==1]['int_rate']], 
                  labels=['Good', 'Default'])
axes[0,1].set_title('Interest Rate vs Default')
axes[0,1].set_ylabel('Interest Rate (%)')

# 3. Annual Income vs Default
axes[0,2].boxplot([df[df['is_default']==0]['annual_inc'], 
                   df[df['is_default']==1]['annual_inc']], 
                  labels=['Good', 'Default'])
axes[0,2].set_title('Annual Income vs Default')
axes[0,2].set_ylabel('Annual Income')
axes[0,2].set_ylim(0, 200000)  # จำกัดแกน y

# 4. DTI vs Default
axes[1,0].boxplot([df[df['is_default']==0]['dti'], 
                   df[df['is_default']==1]['dti']], 
                  labels=['Good', 'Default'])
axes[1,0].set_title('Debt-to-Income Ratio vs Default')
axes[1,0].set_ylabel('DTI')

# 5. Grade vs Default Rate
grade_default = df.groupby('grade')['is_default'].mean().sort_index()
axes[1,1].bar(grade_default.index, grade_default.values)
axes[1,1].set_title('Default Rate by Grade')
axes[1,1].set_ylabel('Default Rate')

# 6. Employment Length vs Default Rate
emp_default = df.groupby('emp_length_years')['is_default'].mean()
axes[1,2].plot(emp_default.index, emp_default.values, marker='o')
axes[1,2].set_title('Default Rate by Employment Length')
axes[1,2].set_xlabel('Employment Length (Years)')
axes[1,2].set_ylabel('Default Rate')

plt.tight_layout()
plt.show()

## 5. Prepare Data for ML

In [ ]:
# เลือก features สำหรับ ML
feature_columns = [
    'loan_amnt', 'int_rate', 'installment', 'annual_inc', 'dti', 
    'emp_length_years', 'home_ownership_encoded', 'verification_status_encoded',
    'purpose_encoded', 'grade_encoded'
]

# ลบแถวที่มีข้อมูลหายไป
df_clean = df[feature_columns + ['is_default']].dropna()

X = df_clean[feature_columns]
y = df_clean['is_default']

print(f"Final dataset: {X.shape[0]} samples, {X.shape[1]} features")
print(f"Default rate: {y.mean():.2%}")

# แบ่งข้อมูล train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"\nTraining set: {X_train.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")

In [ ]:
# Feature Scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# แปลงกลับเป็น DataFrame เพื่อความสะดวก
X_train_scaled = pd.DataFrame(X_train_scaled, columns=feature_columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=feature_columns)

print("Feature scaling completed!")
print("\nFeature statistics after scaling:")
print(X_train_scaled.describe().round(3))

## 6. Model Training และ Evaluation

In [ ]:
# สร้างและฝึก models
models = {
    'Logistic Regression': LogisticRegression(random_state=42, max_iter=1000),
    'Decision Tree': DecisionTreeClassifier(random_state=42, max_depth=10),
    'Random Forest': RandomForestClassifier(random_state=42, n_estimators=100, max_depth=10)
}

# ฝึกและประเมิน
results = {}

for name, model in models.items():
    print(f"\nTraining {name}...")
    
    # ใช้ scaled data สำหรับ Logistic Regression
    if name == 'Logistic Regression':
        model.fit(X_train_scaled, y_train)
        y_pred = model.predict(X_test_scaled)
        y_prob = model.predict_proba(X_test_scaled)[:, 1]
    else:
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        y_prob = model.predict_proba(X_test)[:, 1]
    
    # เก็บผลลัพธ์
    accuracy = accuracy_score(y_test, y_pred)
    results[name] = {
        'model': model,
        'predictions': y_pred,
        'probabilities': y_prob,
        'accuracy': accuracy
    }
    
    print(f"Accuracy: {accuracy:.3f}")
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred))

In [ ]:
# เปรียบเทียบผลลัพธ์
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score

comparison_results = pd.DataFrame()

for name, result in results.items():
    y_pred = result['predictions']
    y_prob = result['probabilities']
    
    metrics = {
        'Accuracy': accuracy_score(y_test, y_pred),
        'Precision': precision_score(y_test, y_pred),
        'Recall': recall_score(y_test, y_pred),
        'F1-Score': f1_score(y_test, y_pred),
        'AUC-ROC': roc_auc_score(y_test, y_prob)
    }
    
    comparison_results[name] = metrics

print("Model Comparison:")
print(comparison_results.round(3))

# แสดงผลในรูปกราฟ
comparison_results.T.plot(kind='bar', figsize=(12, 6))
plt.title('Model Performance Comparison')
plt.ylabel('Score')
plt.xticks(rotation=45)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

## 7. Cross-Validation

In [ ]:
# Cross-validation สำหรับแต่ละ model
from sklearn.model_selection import StratifiedKFold

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cv_results = {}

for name, model in models.items():
    print(f"\nCross-validation for {name}:")
    
    if name == 'Logistic Regression':
        scores = cross_val_score(model, X_train_scaled, y_train, cv=cv, scoring='accuracy')
    else:
        scores = cross_val_score(model, X_train, y_train, cv=cv, scoring='accuracy')
    
    cv_results[name] = scores
    print(f"CV Scores: {scores}")
    print(f"Mean CV Score: {scores.mean():.3f} (+/- {scores.std() * 2:.3f})")

# แสดงผล CV scores
plt.figure(figsize=(10, 6))
plt.boxplot([cv_results[name] for name in models.keys()], 
           labels=list(models.keys()))
plt.title('Cross-Validation Scores')
plt.ylabel('Accuracy')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 8. Feature Importance

In [ ]:
# Feature importance สำหรับ Random Forest
rf_model = results['Random Forest']['model']
feature_importance = pd.DataFrame({
    'feature': feature_columns,
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=False)

print("Feature Importance (Random Forest):")
print(feature_importance)

# แสดงผลในรูปกราฟ
plt.figure(figsize=(10, 6))
sns.barplot(data=feature_importance, x='importance', y='feature')
plt.title('Feature Importance - Random Forest')
plt.xlabel('Importance')
plt.tight_layout()
plt.show()

## สรุป

### ผลลัพธ์การทดลอง
- เปรียบเทียบประสิทธิภาพของ 3 algorithms
- Random Forest มักให้ผลลัพธ์ที่ดีที่สุด
- Features ที่สำคัญ: อัตราดอกเบี้ย, เกรด, DTI

### สิ่งที่เรียนรู้
- Data preprocessing สำคัญมาก
- Cross-validation ช่วยประเมินความเสถียร
- Feature importance ช่วยเข้าใจข้อมูล

### Next: Regression Algorithms